In [1]:
import pickle
import pandas as pd
import os
import torch
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
from ft_sec_key import SECKEY
import logging
import concurrent.futures
import numpy as np
import featurecreations_polars
import polars as pl

In [4]:
data_file = '../data/gru_inputs_kitchen_sink_april.fth'
data = pd.read_feather(data_file)

In [7]:
data = data.query('state == "NSW"')
data.shape

(398315, 25)

In [8]:
data.to_feather('../data/gru_inputs_kitchen_sink_april_nsw.fth')

: 

In [10]:
def generate_dog_stats_simple(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.clone()

    
    new_cols = ['dist_x','box','split_speed','split_margins','margin','MarginTime','RunHomeTime','run_home_speed',
                'first_out','start_pos','pos_change','win','win','win','weight',"RunTime","SplitTimes","StartPrice","StartProb",
        ]


    try:
        df = df.with_columns(pl.col('dist_x').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_dist_last__{rolling_window}'))
    except:
        df = df.with_columns(pl.lit(0).alias(f'{factor}_dist_last__{rolling_window}'))
    try:
        df = df.with_columns(pl.col('box').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_box_last__{rolling_window}'))
    except:
        df = df.with_columns(pl.lit(0).alias(f'{factor}_box_last__{rolling_window}'))

    # df = df_g.with_columns(pl.col('box').cast(int))
    new_cols = [pl.col('speed').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_speed_avg_{rolling_window}'),
    # .alias(f'{factor}_split_speed_v1_{rolling_window}')) _ pl.col('split_speed_v1').rolling_mean(window_size=rolling_window, min_periods=1, closed='left')['split_speed_v1',
    pl.col('split_speed').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_split_speed_avg_{rolling_window}'),
    pl.col('split_margins').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_split_margin_avg_{rolling_window}'),
    pl.mean('margin').rolling(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_margin_avg_{rolling_window}'),
    pl.col('MarginTime').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_margin_time_avg_{rolling_window}'),
    pl.col('RunHomeTime').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_RunHomeTime_{rolling_window}'),
    pl.col('run_home_speed').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_run_home_speed_{rolling_window}'),
    pl.col('first_out').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_first_out_avg_{rolling_window}'),
    pl.col('start_pos').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_pos_out_avg_{rolling_window}'),
    pl.col('pos_change').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_post_change_avg_{rolling_window}'),
    pl.col('win').cum_count().alias(f'{factor}_races_{rolling_window}'),
    pl.col('win').rolling_mean(window_size=1000, min_periods=1, closed='left').alias(f'{factor}_wins_{rolling_window}'),
    pl.col('win').rolling_sum(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_wins_last_{rolling_window}'),
    pl.col('weight').rolling_mean(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_weight_'),
    pl.col("RunTime").rolling_min(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_min_time_'),
    pl.col("SplitTimes").rolling_min(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_min_split_time_'),
    pl.col("StartPrice").rolling_min(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_last_start_price'),
    pl.col("StartProb").rolling_min(window_size=rolling_window, min_periods=1, closed='left').alias(f'{factor}_last_start_prob')]

    df = df_g.map_groups(lambda group: group.with_columns(new_cols))
    
    return(df)




In [14]:
all_results = pd.read_feather(r"./DATA/results-df-merged-prices_cut.fth")


In [15]:
all_results.columns

Index(['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date',
       'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP',
       'Place', 'place', 'margin', 'box', 'speed', 'split_speed',
       'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime',
       'SplitTimes', 'StartPrice', 'DogGrade', 'race_time', 'tracksOnehot',
       'MarginTime', 'run_home_speed_v1', 'run_home_speed', 'split_speed_v1',
       'RunHomeTime', 'PIR', 'PIR_adj', 'start_pos', 'StartProb',
       'dist_round'],
      dtype='object')

In [18]:
all_results.State.unique()

array([None, 'VIC', 'NSW', 'SA', 'QLD', 'NZ', 'NT', 'WA', 'TAS'],
      dtype=object)

In [19]:
STATES = ['VIC', 'NSW', 'SA', 'QLD', 'NZ', 'NT', 'WA', 'TAS']

In [20]:
for state in STATES:
    print(state)
    df = all_results[all_results.State == state]
    df.to_feather(f"./DATA/results-df-merged-prices_cut_{state}.fth")

VIC
NSW
SA
QLD
NZ
NT
WA
TAS


In [5]:
all_results = all_results.query('State == "NZ"')

In [8]:
all_results.to_feather(r"./DATA/results-df-merged-prices_cut_NZ.fth")


In [9]:
dog_form = pd.read_feather(r'./DATA/ft_races_ALL.fth')

In [12]:
dog_form = dog_form.query('State == "NZ"')
dog_form.to_feather(r'./DATA/ft_races_NZ.fth')

In [ ]:
dog_form

In [4]:
all_results = pd.read_feather(r"./DATA/results-df-merged-prices_cut.fth")
# all_results["tracksOnehot"] = [x for x in ohe.transform(all_results[["Track"]])]
form = all_results.sort_values("dateF", ascending=True)
form.loc[form['place']==1, 'margin']=0    

form['win'] = form['place'].apply(lambda x: 1 if x ==1 else 0)
form['count'] = 1
form['box_location'] = form['box'].apply(lambda x: [1,0,0] if x<3 else([0,1,0] if x<7 else [0,0,1]))
form['inside'] = form['box'].apply(lambda x: 1 if x<3 else 0)
form['midfield'] = form['box'].apply(lambda x: 1 if 2<x<7 else 0)
form['wide'] = form['box'].apply(lambda x: 1 if 6<x else 0)

#For OG only

form['first_out'] = form['start_pos'].apply(lambda x: 1 if x==1 else 0)

all_results['RaceId'] = all_results['RaceId'].astype('category')
all_results['DogId'] = all_results['DogId'].astype('category')
all_results['RaceId'] = all_results['RaceId'].astype('category')

# all_results.reset_index(drop=True).to_feather(previous_results_file)

In [5]:
form.query("DogId == '251076188'").head(10)[['dateF','dist_x','place','margin']]

,dateF,dist_x,place,margin
679,2019-12-01,516.0,2.0,2.00
27252,2019-12-31,516.0,4.0,7.75
31065,2020-01-04,520.0,7.0,11.50
36984,2020-01-11,520.0,7.0,13.75
76780,2020-02-24,520.0,6.0,8.50
83413,2020-03-02,520.0,8.0,15.75
95070,2020-03-16,520.0,7.0,12.25
99194,2020-03-20,516.0,6.0,9.75
104324,2020-03-27,400.0,1.0,0.00
108497,2020-04-02,516.0,4.0,5.25


In [6]:
print("starting stats rolling window")


#NEW Groups for functions
form = form.reset_index(drop=True)
form = form.query("DogId == '251076188'")
print(f"{form.shape=}")
# form = form[form['State']=="VIC"].reset_index()
print(f"{form.shape=}")
form['box_g'] = form['box']
form['dist_g'] = form['dist_x']
dog_form = form.groupby(['DogId'], sort=False, as_index=False)

form = pl.from_pandas(form)
dog_form = form.group_by(['DogId'])


print(f"{form.columns=}")

starting stats rolling window
form.shape=(16, 46)
form.shape=(16, 46)
form.columns=['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date', 'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP', 'Place', 'place', 'margin', 'box', 'speed', 'split_speed', 'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime', 'SplitTimes', 'StartPrice', 'DogGrade', 'race_time', 'tracksOnehot', 'MarginTime', 'run_home_speed_v1', 'run_home_speed', 'split_speed_v1', 'RunHomeTime', 'PIR', 'PIR_adj', 'start_pos', 'StartProb', 'dist_round', 'win', 'count', 'box_location', 'inside', 'midfield', 'wide', 'box_g', 'dist_g']


C:\Users\Nick\AppData\Local\Temp\ipykernel_22276\1176839702.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dog_form = form.groupby(['DogId'], sort=False, as_index=False)


In [11]:
dog_form = form.groupby(['DogId'])
form_stats = generate_dog_stats_simple(form, dog_form,rolling_window=1)
print("fin 1")

C:\Users\Nick\AppData\Local\Temp\ipykernel_22276\1764915137.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  dog_form = form.groupby(['DogId'])


TypeError: Expr.rolling() got an unexpected keyword argument 'window_size'

In [13]:
form_stats = form.with_columns(pl.col('margin').rolling_mean(1,closed='left').by_window(pl.col('DogId'))).alias('_margin_avg_1'))

SyntaxError: unmatched ')' (3853317322.py, line 1)

In [9]:
form_stats[['_margin_avg_1']]

_margin_avg_1
f64
2.0
7.75
11.5
13.75
8.5
15.75
12.25
9.75
0.0


In [82]:
print(form.columns)
dist_form = form.group_by(['DogId','dist_round'])
box_form = form.groupby(['DogId','box_g'])
track_box_form = form.groupby(['DogId','box_g','Track'])
track_dist_form = form.groupby(['DogId','dist_g','Track'])
# form = generate_prev_weight(form, dog_form)
# form_stats = generate_dog_stats(form, dog_form,rolling_window=30)
# print("fin 30")
# form_stats = generate_dog_stats(form_stats, dog_form,rolling_window=5)
# print("fin 5")
form_stats = generate_dog_stats_simple(form, dog_form,rolling_window=1)
print("fin 1")
form_stats = generate_dog_stats_simple(form_stats, dist_form,rolling_window=10,factor='dist')
print("fin 10_dist")
form_stats = generate_dog_stats_simple(form_stats, box_form,rolling_window=10,factor='box')
print("fin 10 box")
form_stats = generate_dog_stats_simple(form_stats, track_box_form,rolling_window=10,factor='track_box')
print("fin 10 track_box")
form_stats = generate_dog_stats_simple(form_stats, track_dist_form,rolling_window=10,factor='track_dist')
print("fin 10 track_dist")  

['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date', 'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP', 'Place', 'place', 'margin', 'box', 'speed', 'split_speed', 'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime', 'SplitTimes', 'StartPrice', 'DogGrade', 'race_time', 'tracksOnehot', 'MarginTime', 'run_home_speed_v1', 'run_home_speed', 'split_speed_v1', 'RunHomeTime', 'PIR', 'PIR_adj', 'start_pos', 'StartProb', 'dist_round', 'win', 'count', 'box_location', 'inside', 'midfield', 'wide', 'box_g', 'dist_g']


C:\Users\Nick\AppData\Local\Temp\ipykernel_15976\4158808149.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  box_form = form.groupby(['DogId','box_g'])
C:\Users\Nick\AppData\Local\Temp\ipykernel_15976\4158808149.py:4: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  track_box_form = form.groupby(['DogId','box_g','Track'])
C:\Users\Nick\AppData\Local\Temp\ipykernel_15976\4158808149.py:5: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  track_dist_form = form.groupby(['DogId','dist_g','Track'])


PanicException: UDF failed: 

In [69]:
print(form_stats.columns)

['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date', 'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP', 'Place', 'place', 'margin', 'box', 'speed', 'split_speed', 'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime', 'SplitTimes', 'StartPrice', 'DogGrade', 'race_time', 'tracksOnehot', 'MarginTime', 'run_home_speed_v1', 'run_home_speed', 'split_speed_v1', 'RunHomeTime', 'PIR', 'PIR_adj', 'start_pos', 'StartProb', 'dist_round', 'win', 'count', 'box_location', 'inside', 'midfield', 'wide', 'box_g', 'dist_g', '_dist_last__1', '_box_last__1', '_speed_avg_1', '_split_speed_avg_1', '_split_margin_avg_1', '_margin_avg_1', '_margin_time_avg_1', '_RunHomeTime_1', '_run_home_speed_1', '_first_out_avg_1', '_pos_out_avg_1', '_post_change_avg_1', '_races_1', '_wins_1', '_wins_last_1', '_weight_', '_min_time_', '_min_split_time_', '_last_start_price', '_last_start_prob', 'dist_dist_last__10', 'dist_box_last__10', 'dist_speed_avg_10', 'dist_split_speed

In [70]:
form = form_stats.to_pandas()

In [71]:
star = form.query("DogId == '251076188'").head(10)

In [72]:
star[['dateF','_dist_last__1','dist_x','place','margin','_margin_avg_1']]

,dateF,_dist_last__1,dist_x,place,margin,_margin_avg_1
0,2019-12-01,NaN,NaN,2.0,NaN,NaN
27757,2019-12-31,516.0,400.0,4.0,3.75,7.50
30693,2020-01-04,300.0,300.0,7.0,2.11,3.97
37370,2020-01-11,520.0,520.0,7.0,18.50,6.00
76394,2020-02-24,520.0,520.0,6.0,0.00,7.50
83095,2020-03-02,520.0,520.0,8.0,8.25,15.25
95639,2020-03-16,520.0,450.0,7.0,5.50,0.00
98797,2020-03-20,516.0,516.0,6.0,0.00,8.50
104613,2020-03-27,400.0,400.0,1.0,8.25,20.75
108050,2020-04-02,516.0,400.0,4.0,13.75,4.00
